# Support Vector Machines

Use SVM to seperate the predict land use type

In [1]:
import pandas as pd 
import numpy as np
import pylab as pl
from sklearn import datasets
from sklearn.multiclass import OneVsRestClassifier
from sklearn import svm
from sklearn.grid_search import GridSearchCV
from sklearn import preprocessing
import math
import warnings
import os


warnings.filterwarnings("ignore")

pl.style.use('fivethirtyeight')
pl.rcParams['figure.figsize'] = (16,16)

In [2]:
columns = ['ID', 'GPS_DATETIMESTAMP', 'GPS_LAT', 'GPS_LON', 'GPS_Speed',\
           'GPS_Alt', 'GPS_Sats', 'GPS_Fix', 'GPS_Quality', 'AMB_Temp', \
           'AMB_Humd', 'AMB_Lux', 'AMB_Snd', 'AMB_SndMin', 'AMB_SndMax',\
           'AMB_SndMea', 'RDQ_AcX', 'RDQ_AcXMin', 'RDQ_AcXMax', 'RDQ_AcXMea',\
           'RDQ_AcY', 'RDQ_AcYMin', 'RDQ_AcYMax', 'RDQ_AcYMea', 'RDQ_AcZ',\
           'RDQ_AcZMin', 'RDQ_AcZMax', 'RDQ_AcZMea', 'sensor', 'timestamp',\
           'hourOfDay', 'minuteOfDay', 'minuteStretched', 'LU05_DESC',\
           'LUCODE', 'merge_key', 'day', 'sensor_num', 'sensor_day',\
           'keep_SndMean', 'is_loud', 'is_dark', 'acel', 'bumpflag', 'latlon_match']
predictors = [columns.index('is_loud'), columns.index('is_dark'), columns.index('bumpflag')]

x_train = np.load('../data/x_train.npy')
y_train = np.load('../data/y_train.npy')
x_test = np.load('../data/x_test.npy')
y_test = np.load('../data/y_test.npy')
x_valid = np.load('../data/x_valid.npy')
y_valid = np.load('../data/y_valid.npy')


In [3]:
x_predictors_train = x_train[:,predictors]
x_predictors_test = x_test[:,predictors]
x_predictors_valid = x_valid[:,predictors]


# Section 1 : Prediction using flags
    - is_loud
    - is_dark
    - is_bump

### Training over several types of kernels

In [15]:
#Linear Kernel
bool_clf = OneVsRestClassifier(svm.SVC(kernel='linear', C=1.0, class_weight='balanced'))
bool_clf.fit(x_predictors_train, y_train)

right=1.0*(bool_clf.predict(x_predictors_test)==np.asarray(y_test)).sum()/len(y_test)

print "we successfully(OS) predict the {} percent of data using a linear kernel".format((right)*100)

we successfully(OS) predict the 58.830541631 percent of data using a linear kernel


In [16]:
# RBF kernel with gamma = auto (1/4 in this case)
bool_rbf_clf = OneVsRestClassifier(svm.SVC(kernel='rbf', C=1.0, class_weight='balanced'))
bool_rbf_clf.fit(x_predictors_train, y_train)

right=1.0*(bool_rbf_clf.predict(x_predictors_test)==np.asarray(y_test)).sum()/len(y_test)

print "we successfully(OS) predict the {} percent of data using a rbf kernel".format((right)*100)

we successfully(OS) predict the 58.530994009 percent of data using a rbf kernel


In [22]:
# RBF kernel with gamma = 0.01
bool_rbf_low_clf = OneVsRestClassifier(svm.SVC(kernel='rbf', C=1.0, gamma = 0.01, class_weight='balanced'))
bool_rbf_low_clf.fit(x_predictors_train, y_train)

right=1.0*(bool_rbf_low_clf.predict(x_predictors_test)==np.asarray(y_test)).sum()/len(y_test)

print "we successfully(OS) predict the {} percent of data using a rbf kernel".format((right)*100)

we successfully(OS) predict the 59.1239760362 percent of data using a rbf kernel


In [21]:
# RBF kernel with gamma = 0.5
bool_rbf_high_clf = OneVsRestClassifier(svm.SVC(kernel='rbf', C=1.0, gamma = 0.5, class_weight='balanced'))
bool_rbf_high_clf.fit(x_predictors_train, y_train)

right=1.0*(bool_rbf_high_clf.predict(x_predictors_test)==np.asarray(y_test)).sum()/len(y_test)

print "we successfully(OS) predict the {} percent of data using a rbf kernel".format((right)*100)

#Caveat gamma higher than 1/num_features risks overfitting

we successfully(OS) predict the 61.1382809634 percent of data using a rbf kernel


In [17]:
bool_poly_clf = OneVsRestClassifier(svm.SVC(kernel='poly', C=1.0, class_weight='balanced'))
bool_poly_clf.fit(x_predictors_train, y_train)

right=1.0*(bool_poly_clf.predict(x_predictors_test)==np.asarray(y_test)).sum()/len(y_test)

print "we successfully(OS) predict the {} percent of data using a poly kernel".format((right)*100)

we successfully(OS) predict the 53.1116273383 percent of data using a poly kernel


In [7]:
os.system('say "Completed prediction for flags"');

# Section 2 : Prediction using standard normal variables
    - Sound
    - Light
    - Z axis acceleration

In [4]:
#transforming sound values to standard normal
std_scaler = preprocessing.StandardScaler()
std_norm_sound_train = std_scaler.fit_transform(x_train[:,columns.index('AMB_SndMea')])
std_norm_sound_valid = std_scaler.fit_transform(x_valid[:,columns.index('AMB_SndMea')])
std_norm_sound_test = std_scaler.fit_transform(x_test[:,columns.index('AMB_SndMea')])


In [5]:
#transforming light values to standard normal
std_norm_light_train = std_scaler.fit_transform(x_train[:,columns.index('AMB_Lux')])
std_norm_light_valid = std_scaler.fit_transform(x_valid[:,columns.index('AMB_Lux')])
std_norm_light_test = std_scaler.fit_transform(x_test[:,columns.index('AMB_Lux')])


In [6]:
#transforming z axis accelerometer values to standard normal
std_norm_bump_train = std_scaler.fit_transform(x_train[:,columns.index('RDQ_AcZMea')])
std_norm_bump_valid = std_scaler.fit_transform(x_valid[:,columns.index('RDQ_AcZMea')])
std_norm_bump_test = std_scaler.fit_transform(x_test[:,columns.index('RDQ_AcZMea')])


In [7]:
# Dropping sound flag (1st column) 
x_sound_predictors_train = x_predictors_train[:,1:]
x_sound_predictors_test = x_predictors_test[:,1:]
x_sound_predictors_valid = x_predictors_valid[:,1:]

# Adding sound std normal values
x_sound_predictors_train = np.insert(x_sound_predictors_train, 0,std_norm_sound_train, axis = 1)
x_sound_predictors_test = np.insert(x_sound_predictors_test, 0,std_norm_sound_test, axis = 1)
x_sound_predictors_valid = np.insert(x_sound_predictors_valid, 0,std_norm_sound_valid, axis = 1)
#TODO : Include accelerator values on all axes 

### Training over a linear kernel

In [8]:
# Test train and validation with std norm variables instead of flag

x_std_predictors_train = np.array([std_norm_sound_train, std_norm_bump_train, std_norm_light_train]).T
x_std_predictors_test = np.array([std_norm_sound_test, std_norm_bump_test, std_norm_light_test]).T
x_std_predictors_valid = np.array([std_norm_sound_valid, std_norm_bump_valid, std_norm_light_valid]).T


In [9]:
clf = OneVsRestClassifier(svm.SVC(kernel='linear', C=1.0, class_weight='balanced'))
clf.fit(x_std_predictors_train, y_train)

right=1.0*(clf.predict(x_std_predictors_test)==np.asarray(y_test)).sum()/len(y_test)

print "we successfully(OS) predict the {} percent of data using a linear kernel".format((right)*100)

os.system('say "Completed classification using standard normal variables"');

we successfully(OS) predict the 52.2129844724 percent of data using a linear kernel


In [11]:
pd.unique(clf.predict(x_std_predictors_test))

array([1, 2, 0, 3])

In [10]:
rbf_clf = OneVsRestClassifier(svm.SVC(kernel='rbf', C=1.0, class_weight='balanced'))
rbf_clf.fit(x_std_predictors_train, y_train)

right=1.0*(clf.predict(x_std_predictors_test)==np.asarray(y_test)).sum()/len(y_test)

print "we successfully(OS) predict the {} percent of data using a linear kernel".format((right)*100)

os.system('say "Completed classification using standard normal variables"');

we successfully(OS) predict the 52.2129844724 percent of data using a linear kernel


In [12]:
pd.unique(rbf_clf.predict(x_std_predictors_test))

array([3, 0, 2, 1])

In [13]:
poly_clf = OneVsRestClassifier(svm.SVC(kernel='poly', C=1.0, class_weight='balanced'))
poly_clf.fit(x_predictors_train, y_train)

right=1.0*(poly_clf.predict(x_predictors_test)==np.asarray(y_test)).sum()/len(y_test)

print "we successfully(OS) predict the {} percent of data using a poly kernel".format((right)*100)

we successfully(OS) predict the 53.1116273383 percent of data using a poly kernel


In [14]:
pd.unique(poly_clf.predict(x_std_predictors_test))

array([0, 2, 1, 3])

# Section 3 : Tuning Parameters

## Tuning for flags

In [ ]:
tuned_parameters = [ {'kernel': ['linear'], 'C': [0.1, 1, 10, 100, 1000]} ]

validate_clf = GridSearchCV(svm.SVC(C=1, class_weight='balanced'), tuned_parameters, cv=5,
                       scoring='%s_weighted' % 'recall')
validate_clf.fit(x_predictors_train, y_train)

validate_clf.best_params_

C = np.linspace(-5,50,10)
C=[math.exp(i) for i in C]
OS_validation=[]
for c in C:
    clf = svm.SVC(kernel='linear',C=c) 
    clf.fit(x_predictors_train, y_train)
    right=1.0*(clf.predict(x_predictors_valid)==np.asarray(y_valid)).sum()/len(y_valid)
    OS_validation.append(right)
    
temp=pd.DataFrame([C,OS_validation]).T
ind=len(temp.loc[temp.iloc[:,1]==temp.iloc[:,1].max()])/2 
C_opt=temp.loc[temp.iloc[:,1]==temp.iloc[:,1].max()].iloc[ind,0]


C=[math.log(y,10) for y in C]# for a better graph
pylab.plot(C,OS_validation,'b',)
pylab.legend(loc='upper right')
pylab.ylabel('Accuracy')
pylab.xlabel('log(C)')
pylab.show()

print("The optimal C we found is:{}".format(C_opt)) 
''' '''

In [ ]:
os.system('say "tuning parameters done"');

In [25]:
np.logspace(-9, 3, 13)

array([  1.00000000e-09,   1.00000000e-08,   1.00000000e-07,
         1.00000000e-06,   1.00000000e-05,   1.00000000e-04,
         1.00000000e-03,   1.00000000e-02,   1.00000000e-01,
         1.00000000e+00,   1.00000000e+01,   1.00000000e+02,
         1.00000000e+03])

In [27]:
np.linspace?

In [ ]:
{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [1, 10, 100, 1000]},